In [1]:
import os
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
from dotenv import load_dotenv

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
class Website:
    """
    A utility class to represent a Website that we have scraped
    """
    url: str
    title: str
    text: str

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [4]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [5]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "The contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [6]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [7]:

def summarize(url):
    website = Website(url)
    messages = messages_for(website)
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content
    

In [8]:

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [10]:
display_summary("https://icfoss.in/")

# Summary of ICFOSS Website

ICFOSS (International Centre for Free and Open Source Software) is an autonomous organization initiated by the Government of Kerala in 2009. The center aims to promote free and open-source software technologies for widespread use, encouraging collaboration between industry, academia, communities, and government. It focuses on bridging the gap in the FOSS ecosystem through projects, research, training, and capacity building.

## Key Areas of Focus:
- **Research and Development**: Promoting innovation within the FOSS sector.
- **FOSS Training**: Offering workshops and training programs designed to enhance knowledge and skills in open-source technologies.
- **Entrepreneurship**: Supporting startups with the ambition to excel in the FOSS landscape.
- **Community Engagement**: Collaborating with international FOSS communities to foster global partnerships.

## Recent News & Announcements:
1. **Call for Final Year MCA/MSc Project Proposals 2024-2025**: An invitation issued for project proposals from students in relevant fields.
2. **Upcoming Events**:
   - **Third International Conference on FOSS Approaches towards Computational Intelligence and Language Technology**: Scheduled for Jan 22, 2025.
   - **Post Conference Workshop on Open Source Tools for Natural Language Processing**: Set for Jan 25, 2025.

## Vision and Mission:
ICFOSS aims to lead in the research and development of the open-source model, facilitating sustainable development and economic growth in the region.

For more detailed information about their initiatives, projects, or events, you can visit their official site at [ICFOSS](https://www.icfoss.in).